## Water quality in Estonian public pools

Data courtesy of the Estonian National Health Board:

Public data of regular water test procedures: http://vtiav.sm.ee/index.php/opendata/ujulad.xml

Public data about pools (name, address, capacity, details) http://vtiav.sm.ee/index.php/opendata/ujula_valdajad.xml

-----------------------------------------------------
Using ElementTree and Pandas.
Drilling down to the lowest XML node level that I need for the data (Using for loops and IF statements because it worked). 

Then I write the results into a empty dictionary of lists that act as columns.
This tranforms the data into columns, with parent node values filling in empty cells.
The dictionary I then transform into a Pandas dataframe. 

In [3]:
import urllib
import pandas as pd
import xml.etree.cElementTree as et

In [1]:
url = 'http://vtiav.sm.ee/index.php/opendata/basseini_veeproovid.xml'

parsed_data = et.parse(urllib.request.urlopen(url))

proc_data_dict = {
    'pool_id': [],
    'pool_nm': [],
    'test_dt': [],
    'proc_id': [],
    'proc_rs': [],
    'proc_test_nm': [],
    'proc_test_rs': [],
    'proc_test_val': [],
    'proc_test_unit': []
}

tree = parsed_data
root = tree.getroot()

for i in root:
    for i2 in i:
        for i3 in i2:
            if i3.find('hinnang') != None:
                for i4 in i3:
                    for i5 in i4:
                        if i5.find('id') != None:
                            proc_data_dict['proc_id'].append(i3.find('katseprotokolli_number').text)
                            proc_data_dict['proc_rs'].append(i3.find('hinnang').text)
                            proc_data_dict['proc_test_nm'].append(i5.find('nimetus').text)
                            proc_data_dict['proc_test_rs'].append(i5.find('hinnang').text)
                            proc_data_dict['proc_test_val'].append(i5.find('sisaldus').text)
                            proc_data_dict['proc_test_unit'].append(i5.find('yhik').text)
                            proc_data_dict['pool_id'].append(i.find('bassein_id').text)
                            proc_data_dict['pool_nm'].append(i.find('bassein').text)
                            proc_data_dict['test_dt'].append(i.find('proovivotu_aeg').text)
                            

proc_data_df = pd.DataFrame(proc_data_dict)

proc_data_df.head()

,pool_id,pool_nm,test_dt,proc_id,proc_rs,proc_test_nm,proc_test_rs,proc_test_val,proc_test_unit
0,344,Laine SPA bassein,01.10.2020 09:00,KL2015/V48283K,ei vasta nõuetele,Seotud kloor (<31 °C),ei vasta nõuetele,1.4,mg/l
1,344,Laine SPA bassein,01.10.2020 09:00,KL2015/V48283K,ei vasta nõuetele,Hägusus,vastab nõuetele,1.0,NHÜ
2,344,Laine SPA bassein,01.10.2020 09:00,KL2015/V48283K,ei vasta nõuetele,Nitraatioon,vastab nõuetele,1.2,mg/l
3,344,Laine SPA bassein,01.10.2020 09:00,KL2015/V48283K,ei vasta nõuetele,Ammooniumioon,vastab nõuetele,0.05,mg/l
4,344,Laine SPA bassein,01.10.2020 09:00,KL2015/V48283K,ei vasta nõuetele,Vaba kloor,vastab nõuetele,0.6,mg/l


In [10]:
proc_data_df.to_csv('est_pool_proc_data.csv')

###### Next steps:

* get data about pools too (pool complex name, address, capacity etc), add to proc data
* vizualise data with pandas (percent of failed test per pool?)

In [61]:
url_02 = 'http://vtiav.sm.ee/index.php/opendata/ujulad.xml'
parsed_data_02 = et.parse(urllib.request.urlopen(url_02))

tree_02 = parsed_data_02
root_02 = tree_02.getroot()

pool_data_dict = {
    'est_id': [],
    'est_nm': [],
#    'est_adr': [],
#    'est_tp': [],
#    'est_usr_am': [],
#    'est_own_nm': [],
    'pool_id': [],
    'pool_nm': []
}

for i in root_02:
    for i2 in i:
        for i3 in i2:
            for i4 in i3:
                if i3.find('id') is not None:
                    pool_data_dict['pool_id'].append(i3.find('id').text)
                    pool_data_dict['pool_nm'].append(i3.find('nimetus').text)
                    pool_data_dict['est_id'].append(i.find('id').text)
                    pool_data_dict['est_nm'].append(i.find('nimetus').text)

pool_data_df = pd.DataFrame(pool_data_dict)

In [62]:
pool_data_df.to_csv('est_pool_data.csv')